In [1]:
import ee
import geopandas
import datetime
import numpy as np
import pandas as pd
import folium
from ast import literal_eval
from shapely import wkt
from shapely.geometry import box, Point, LineString, Polygon, MultiLineString
from shapely.ops import split
import math
import warnings
import os
from functools import reduce

In [2]:
# ee.Initialize()

#### Run `Sample_Sentinel2_by_Country.py`

* `Global_Samples_Balanced_n100_m250_s210_attempt1.csv`
    * Pulled every country except for RUS (RUS data was incomplete due to exceeding GEE 5000 image limit)
    * The output data is saved in the `/data/remote/Sentinel2_samples_Cloud50_n100_m250_s210_attempt1_incomplete_RUS` folder
    * RUS data was pulled again using `Global_Samples_Balanced_n100_m200_s210.csv` and saved in the `/data/remote/Sentinel2_samples_Cloud50_n100_m200_s210_incomplete_RUS` folder but it's incomplete again because I hit the account limit.
    * The aggregate data is saved in `/data/Sample_Sentinel_2018_Cloud50_n100_m250_s210_attempt1.csv`
* `Global_Samples_Balanced_n100_m200_s211.csv`
    * Pulled every country except that RUS, CHN, and USA were incomplete because I hit the account limit.
    * The output data is saved in the `/data/remote/Sentinel2_samples_Cloud50_n100_m200_s211_incomplete_RUS_CHN_USA` folder
    * The aggregate data is saved in `/data/Sample_Sentinel_2018_Cloud50_n100_m200_s211_incomplete_RUS_CHN_USA.csv`
* `Global_Samples_Balanced_n200_m50_s213.csv`
    * This only downloaded sampled irrigated land data
    * The output data is saved in the `/data/remote/Sentinel2_samples_Cloud50_n200_m50_s213` folder
    * The aggregate data is saved in `/data/Sample_Sentinel_2018_Cloud50_n200_m50_s213.csv`

In [3]:
sample_file = '../data/Global_Samples_Balanced_n200_m50_s213.csv'

In [4]:
world_df = pd.read_csv(sample_file, index_col=[0])
world_df.head()
world_df['geometry'] = world_df['geometry'].apply(wkt.loads)
world_df['samples'] = world_df['samples'].apply(literal_eval)
world_df['n_samples'] = world_df.apply(lambda row: len(row['samples']), axis=1)
world_gdf = geopandas.GeoDataFrame(world_df, geometry='geometry')
world_gdf = world_gdf[(world_gdf['n_samples']>0)]
world_gdf.head()

,country_code,geometry,area,samples,n_samples
1,FJI,"POLYGON ((178.12557 -17.50481, 178.37360 -17.3...",0.983740,"[[177.50428304875032, -17.681708374019223]]",1
3,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",41.629232,"[[35.660584639565975, -2.7239110740946204], [3...",6
4,TZA,"POLYGON ((39.20222 -4.67677, 38.74054 -5.90895...",34.672732,"[[37.56683623717293, -7.608781878929718], [37....",4
6,CAN,"POLYGON ((-70.19171 47.03804, -68.65000 48.300...",16.730885,"[[-63.59254938357725, 45.81992037628136], [-64...",6
7,CAN,"POLYGON ((-63.25471 44.67014, -64.24656 44.265...",2.471299,"[[-65.09463983153698, 44.94090517796168]]",1


In [6]:
country_lst = list(set(world_gdf['country_code'].tolist()))
country_lst.sort()
country_lst

['AFG',
 'AGO',
 'ALB',
 'ARG',
 'ARM',
 'AUS',
 'AUT',
 'AZE',
 'BEL',
 'BEN',
 'BFA',
 'BGD',
 'BGR',
 'BIH',
 'BLR',
 'BOL',
 'BRA',
 'BWA',
 'CAN',
 'CHE',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COL',
 'CUB',
 'CZE',
 'DEU',
 'DJI',
 'DNK',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ESP',
 'EST',
 'ETH',
 'FIN',
 'FJI',
 'GAB',
 'GBR',
 'GEO',
 'GHA',
 'GIN',
 'GMB',
 'GNB',
 'GRC',
 'GUY',
 'HND',
 'HRV',
 'HTI',
 'HUN',
 'IDN',
 'IND',
 'IRL',
 'IRN',
 'IRQ',
 'ITA',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KHM',
 'KOR',
 'LBN',
 'LBR',
 'LBY',
 'LKA',
 'LSO',
 'LTU',
 'LVA',
 'MAR',
 'MDA',
 'MDG',
 'MEX',
 'MKD',
 'MLI',
 'MMR',
 'MNG',
 'MOZ',
 'MWI',
 'MYS',
 'NAM',
 'NCL',
 'NER',
 'NGA',
 'NIC',
 'NLD',
 'NPL',
 'NZL',
 'OMN',
 'PAK',
 'PAN',
 'PER',
 'PHL',
 'PNG',
 'POL',
 'PRK',
 'PRT',
 'PRY',
 'ROU',
 'RUS',
 'RWA',
 'SAU',
 'SDN',
 'SEN',
 'SOM',
 'SRB',
 'SSD',
 'SVK',
 'SWE',
 'SWZ',
 'SYR',
 'TCD',
 'TGO',
 'THA',
 'TJK',
 'TKM',
 'TLS',
 'TUN',
 'TUR',
 'TWN',


In [8]:
for i in country_lst:
    # Run on IBM Cloud
    print('nohup python3 -u /git/Sample_Sentinel2_by_Country.py {} 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_{}.log &'.format(i, i))

nohup python3 -u /git/Sample_Sentinel2_by_Country.py AFG 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_AFG.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py AGO 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_AGO.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py ALB 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_ALB.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py ARG 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_ARG.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py ARM 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_ARM.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py AUS 2018 --resolution 30 > /git/GEE/Sentinel2_samples_Cloud50_n200_m50_s213/Sentinel2_AUS.log &
nohup python3 -u /git/Sample_Sentinel2_by_Country.py AUT 2018 --resolution 30 > /git/GEE/Sentinel2_samples

#### Check the output from `Sample_Sentinel2_by_Country.py`

In [14]:
# output_dir = '../data/remote/Sentinel2_samples_Cloud50_n100_m250_s210_attempt1_incomplete_RUS'
# output_dir = '../data/remote/Sentinel2_samples_Cloud50_n100_m200_s210_incomplete_RUS'
# output_dir = '../data/remote/Sentinel2_samples_Cloud50_n100_m200_s211_incomplete_RUS_CHN_USA'
output_dir = '../data/remote/Sentinel2_samples_Cloud50_n200_m50_s213'

* Check if GEE limit is reached when processing certain countries

In [15]:
error_msg = 'ee.ee_exception.EEException: Collection query aborted after accumulating over 5000 elements.'

In [16]:
for f in os.listdir(output_dir):
    if f.endswith(".log"):
        with open(os.path.join(output_dir, f)) as ff:
            if error_msg in ff.read():
                print('===== {} ===='.format(f))
                print("True")

In [17]:
error_msg = 'ee.ee_exception.EEException: Quota exceeded for quota metric \'Number of read requests\' and limit \'Number of read requests per minute per user\' of service \'earthengine.googleapis.com\' for consumer'

In [18]:
for f in os.listdir(output_dir):
    if f.endswith(".log"):
        with open(os.path.join(output_dir, f)) as ff:
            if error_msg in ff.read():
                print('===== {} ===='.format(f))
                print("True")

* Create a metadata for all the .csv file

In [19]:
csv_lst = [f.split('_') for f in os.listdir(output_dir) if f.endswith(".csv")]
csv_df = pd.DataFrame(csv_lst, columns=('country_code', 'segment', 'YYYYMM'))
# Remove .csv in YYYYMM
csv_df['YYYYMM'] = csv_df.apply(lambda row: row['YYYYMM'].split('.')[0], axis=1)
csv_df.head()

,country_code,segment,YYYYMM
0,AFG,0,201801
1,AFG,0,201802
2,AFG,0,201803
3,AFG,0,201804
4,AFG,0,201805


In [20]:
csv_df['country_segment_code'] = csv_df.apply(lambda row: row['country_code']+'_'+row['segment'], axis=1)
csv_df['value'] = 1
# YYYYMM is actually the column level names rather than index
# Reference: https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index
csv_pivot = csv_df.pivot(index='country_segment_code', columns='YYYYMM', values='value') \
                    .add_prefix('YM') \
                    .reset_index()
csv_pivot

YYYYMM,country_segment_code,YM201801,YM201802,YM201803,YM201804,YM201805,YM201806,YM201807,YM201808,YM201809,YM201810,YM201811,YM201812
0,AFG_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AFG_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AGO_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AGO_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,ALB_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,ARG_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,ARG_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,ARG_2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,ARG_3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,ARG_4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Fine rows with NaN

In [21]:
is_NaN = csv_pivot.isnull()
row_has_NaN = is_NaN.any(axis=1)
csv_pivot[row_has_NaN]

YYYYMM,country_segment_code,YM201801,YM201802,YM201803,YM201804,YM201805,YM201806,YM201807,YM201808,YM201809,YM201810,YM201811,YM201812
92,CAN_25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
181,FIN_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
334,RUS_28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
344,RUS_37,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
345,RUS_38,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
350,RUS_42,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
351,RUS_43,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
352,RUS_44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
432,USA_36,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


#### Merge all sample data together

In [22]:
country_segment_lst = list(set(csv_df['country_segment_code'].tolist()))
country_segment_nan_lst = csv_pivot[row_has_NaN]['country_segment_code'].tolist()

sentinel_sample_df = pd.DataFrame()
for csc in country_segment_lst:
    if csc in country_segment_nan_lst:
        continue
    print(csc)
    country_sentinel2_lst = [f for f in os.listdir(output_dir) if f.endswith(".csv") and csc+'_' in f]
    if len(country_sentinel2_lst) == 0:
        continue
    # Create a dictionary of all csv pertaining to a country segment in pd.DataFrame
    country_sentinel2_df = dict()
    for f in country_sentinel2_lst:
        temp_df = pd.read_csv(os.path.join(output_dir, f))
        # Drop Unamed column
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]
        country_sentinel2_df[f.split('.')[0]] = temp_df

    country_df_lst = list(country_sentinel2_df.values())
    # Only merge using lat and lon
    #     When an image is unavailable, VI as well as landcover are unavailable
    #     So landcover should be the same if they are available in the data
    country_merge_raw_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), country_df_lst)

    # Create a df that merges all landcover columns into 1
    country_landcover_lst = country_merge_raw_df.loc[:, country_merge_raw_df.columns.str.contains('^landcover')].values.tolist()
    country_landcover_nodup_lst = [set(i) for i in country_landcover_lst]
    country_landcover_no_nan_lst = []
    for i in country_landcover_nodup_lst:
        new_loop = 1
        for j in list(i):
            # NaN in pd is converted to nan (float) in the set or list
            #     Reference: https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values
            if not math.isnan(j):
                country_landcover_no_nan_lst.append(j)
                new_loop = 0
                break
        if new_loop == 1:
            country_landcover_no_nan_lst.append(None)
    country_landcover_df = pd.DataFrame(country_landcover_no_nan_lst, columns=['landcover'])
    # Merge no landcover with landcover df
    country_merge_no_landcover_df = country_merge_raw_df.loc[:, ~country_merge_raw_df.columns.str.contains('^landcover')]
    country_merge_df = pd.merge(country_landcover_df, country_merge_no_landcover_df, left_index=True, right_index=True)
    country_merge_df['country_segment_code'] = csc
    sentinel_sample_df = sentinel_sample_df.append(country_merge_df, ignore_index=True, sort=True)

PHL_2
USA_23
LBY_0
RUS_33
CAN_2
RUS_11
AUS_16
USA_35
IND_6
AGO_1
MEX_5
MOZ_1
USA_26
TGO_0
RUS_47
RUS_21
TUR_2
USA_24
LSO_0
MDG_0
BRA_3
BRA_17
KEN_0
BRA_4
CAN_10
CAN_36
EGY_1
CHN_6
RUS_26
AUS_9
DZA_3
CAN_15
CHN_23
JPN_0
IND_8
IDN_0
RUS_14
KAZ_4
CHN_14
USA_12
USA_4
AFG_0
MMR_1
COL_1
MOZ_0
KAZ_10
CAN_28
ARG_7
NGA_1
DEU_0
GRC_0
SRB_0
AUS_20
BRA_0
AUS_23
MEX_2
NER_1
CHN_0
COG_0
EGY_2
IND_3
MEX_6
SDN_0
CHN_17
PHL_3
CHN_5
AUS_14
HND_0
CAN_6
DZA_0
GAB_0
BRA_15
COD_1
CMR_0
CHL_1
MNG_1
TUN_0
KAZ_5
USA_19
MNG_4
AUS_8
USA_30
RUS_0
TZA_0
MEX_8
USA_7
CAN_14
MDG_1
RUS_45
RUS_12
IND_4
MWI_0
CAN_35
IDN_4
CHN_9
GEO_0
MEX_0
PNG_0
RUS_20
RUS_8
POL_0
ARG_3
ARG_2
RUS_1
GIN_0
BRA_14
AUS_7
CAN_16
CAN_5
USA_3
ARG_0
AUS_25
CAN_7
MLI_1
SAU_2
AUS_2
CHN_22
USA_28
SWE_0
TWN_0
ARM_0
RUS_23
TLS_0
CHN_27
PRK_0
IRN_2
USA_34
BGD_0
IDN_1
BGR_0
NCL_0
CHN_10
OMN_0
AUS_22
DNK_0
KAZ_6
MDA_0
RUS_34
BRA_7
IND_9
AUS_4
AUS_24
CHN_18
KAZ_11
SAU_1
CHN_3
RUS_18
RUS_41
ARG_4
PER_0
RUS_19
UZB_0
EST_0
LBN_0
KAZ_8
IRN_3
IND_7
NZL_0
ETH

In [23]:
is_NaN = sentinel_sample_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
print(sentinel_sample_df[row_has_NaN].shape)
print(sentinel_sample_df.shape)

(1058, 40)
(3625, 40)


In [24]:
# sentinel_sample_df.to_csv('../data/Sample_Sentinel_2018_Cloud50_n200_m50_s213.csv')